In [1]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [4]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
june_results = session.query(Measurement.date, Measurement.tobs).filter(extract('month', Measurement.date) == '6')


In [5]:
#  3. Convert the June temperatures to a list.

june_date = [result[0] for result in june_results]
june_temp = [result[1] for result in june_results]
#print(june_temp)

In [6]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
june_df = pd.DataFrame(june_results, columns=['Date','June Temps'])

print(june_df)

            Date  June Temps
0     2010-06-01        78.0
1     2010-06-02        76.0
2     2010-06-03        78.0
3     2010-06-04        76.0
4     2010-06-05        77.0
...          ...         ...
1695  2017-06-26        79.0
1696  2017-06-27        74.0
1697  2017-06-28        74.0
1698  2017-06-29        76.0
1699  2017-06-30        75.0

[1700 rows x 2 columns]


In [7]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
june_df.describe()

,June Temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [8]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
dec_results = session.query(Measurement.date, Measurement.tobs).filter(extract('month', Measurement.date) == '12')

In [9]:
# 7. Convert the December temperatures to a list.
dec_date = [result[0] for result in dec_results]
dec_temp = [result[1] for result in dec_results]
#print(dec_temp)

In [10]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_df = pd.DataFrame(dec_results, columns=['Date','December Temps'])

print(dec_df)

            Date  December Temps
0     2010-12-01            76.0
1     2010-12-03            74.0
2     2010-12-04            74.0
3     2010-12-06            64.0
4     2010-12-07            64.0
...          ...             ...
1512  2016-12-27            71.0
1513  2016-12-28            71.0
1514  2016-12-29            69.0
1515  2016-12-30            65.0
1516  2016-12-31            65.0

[1517 rows x 2 columns]


In [11]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_df.describe()

,December Temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


In [ ]:
## EXTRA QUERIES ##

In [12]:
# Retrieve the precipation for month of June, save into list and add column to june DataFrame.

june_prcp_results = session.query(Measurement.prcp).filter(extract('month', Measurement.date) == '6')
june_prcp = [result for result in june_prcp_results]
june_df['June Prcps'] = pd.DataFrame(june_prcp_results, columns=['June Prcps'])
print(june_df)

            Date  June Temps  June Prcps
0     2010-06-01        78.0        0.00
1     2010-06-02        76.0        0.01
2     2010-06-03        78.0        0.00
3     2010-06-04        76.0        0.00
4     2010-06-05        77.0        0.00
...          ...         ...         ...
1695  2017-06-26        79.0        0.02
1696  2017-06-27        74.0        0.10
1697  2017-06-28        74.0        0.02
1698  2017-06-29        76.0        0.04
1699  2017-06-30        75.0        0.20

[1700 rows x 3 columns]


In [13]:
# Set index on 'Date' for June DataFrame and diplays the average, minimum and maximum temperatures of June for each year. 

june_df['Date'] = pd.to_datetime(june_df['Date'])
june_df.set_index(june_df['Date'], inplace=True)
grouped_multiple_june_temp = june_df.groupby(june_df['Date'].dt.year).agg({'June Temps': ['mean', 'min', 'max']})
grouped_multiple_june_temp.columns = ['avg_tmp_june', 'min_temp_june', 'max_temp_june']
grouped_multiple_june_temp = grouped_multiple_june_temp.reset_index()
print(grouped_multiple_june_temp)

   Date  avg_tmp_june  min_temp_june  max_temp_june
0  2010     74.925620           67.0           85.0
1  2011     73.938326           65.0           82.0
2  2012     74.000000           68.0           81.0
3  2013     74.599078           66.0           81.0
4  2014     75.027907           67.0           84.0
5  2015     74.990148           66.0           81.0
6  2016     75.175258           64.0           81.0
7  2017     77.219895           71.0           83.0


In [14]:
# Set index on 'Date' for June DataFrame and diplays the average, minimum and maximum precipitatons of June for each year. 

june_df['Date'] = pd.to_datetime(june_df['Date'])
june_df.set_index(june_df['Date'], inplace=True)
grouped_multiple_june_prcp = june_df.groupby(june_df['Date'].dt.year).agg({'June Prcps': ['mean', 'min', 'max']})
grouped_multiple_june_prcp.columns = ['avg_prcp_june', 'min_prcp_june', 'max_prcp_june']
grouped_multiple_june_prcp = grouped_multiple_june_prcp.reset_index()
print(grouped_multiple_june_prcp)

   Date  avg_prcp_june  min_prcp_june  max_prcp_june
0  2010       0.042241            0.0           0.88
1  2011       0.240142            0.0           4.43
2  2012       0.097062            0.0           1.84
3  2013       0.144195            0.0           2.80
4  2014       0.124372            0.0           1.45
5  2015       0.125160            0.0           2.47
6  2016       0.212312            0.0           1.70
7  2017       0.120000            0.0           1.69


In [15]:
# Calculate and print out the temperature and precipitation summary statistics for the June DataFrame
june_df.describe()

,June Temps,June Prcps
count,1700.000000,1574.000000
mean,74.944118,0.136360
std,3.257417,0.335731
min,64.000000,0.000000
25%,73.000000,0.000000
50%,75.000000,0.020000
75%,77.000000,0.120000
max,85.000000,4.430000


In [16]:
# Retrieve the precipation for month of December, save into list and add column to December DataFrame.

dec_prcp_results = session.query(Measurement.prcp).filter(extract('month', Measurement.date) == '12')
dec_prcp = [result for result in dec_prcp_results]
dec_df['December Prcps'] = pd.DataFrame(dec_prcp_results, columns=['December Prcps'])
print(dec_df)

            Date  December Temps  December Prcps
0     2010-12-01            76.0            0.04
1     2010-12-03            74.0            0.00
2     2010-12-04            74.0            0.00
3     2010-12-06            64.0            0.00
4     2010-12-07            64.0            0.00
...          ...             ...             ...
1512  2016-12-27            71.0            0.14
1513  2016-12-28            71.0            0.14
1514  2016-12-29            69.0            1.03
1515  2016-12-30            65.0            2.37
1516  2016-12-31            65.0            0.90

[1517 rows x 3 columns]


In [17]:
# Set index on 'Date' for December DataFrame and diplays the average temperature of December for each year. 

dec_df['Date'] = pd.to_datetime(dec_df['Date'])
dec_df.set_index(dec_df['Date'], inplace=True)
grouped_multiple_dec_temp = dec_df.groupby(dec_df['Date'].dt.year).agg({'December Temps': ['mean', 'min', 'max']})
grouped_multiple_dec_temp.columns = ['temp_avg_dec', 'temp_min_dec', 'temp_max_dec']
grouped_multiple_dec_temp = grouped_multiple_dec_temp.reset_index()
print(grouped_multiple_dec_temp)

   Date  temp_avg_dec  temp_min_dec  temp_max_dec
0  2010     70.208511          58.0          80.0
1  2011     70.820628          63.0          77.0
2  2012     71.188073          61.0          80.0
3  2013     71.094017          63.0          83.0
4  2014     69.896861          56.0          81.0
5  2015     73.423913          65.0          82.0
6  2016     71.130000          60.0          78.0


In [18]:
# Set index on 'Date' for June DataFrame and diplays the average, minimum and maximum precipitatons of December for each year. 

dec_df['Date'] = pd.to_datetime(dec_df['Date'])
dec_df.set_index(dec_df['Date'], inplace=True)
grouped_multiple_dec_prcp = dec_df.groupby(dec_df['Date'].dt.year).agg({'December Prcps': ['mean', 'min', 'max']})
grouped_multiple_dec_prcp.columns = ['prcp_avg_dec', 'prcp_min_dec', 'prcp_max_dec']
grouped_multiple_dec_prcp = grouped_multiple_dec_prcp.reset_index()
print(grouped_multiple_dec_prcp)

   Date  prcp_avg_dec  prcp_min_dec  prcp_max_dec
0  2010      0.459087           0.0          5.04
1  2011      0.201581           0.0          6.42
2  2012      0.089604           0.0          2.25
3  2013      0.169014           0.0          2.40
4  2014      0.188439           0.0          4.03
5  2015      0.169506           0.0          2.21
6  2016      0.199494           0.0          2.37


In [19]:
# Calculate and print out the temperature and precipitation summary statistics for the June DataFrame
dec_df.describe()

,December Temps,December Prcps
count,1517.000000,1405.000000
mean,71.041529,0.216819
std,3.745920,0.541399
min,56.000000,0.000000
25%,69.000000,0.000000
50%,71.000000,0.030000
75%,74.000000,0.150000
max,83.000000,6.420000


In [20]:
# Retrieve the stations for month of June, save into list and create a DataFrame.

s_june_results = session.query(Measurement.date, Measurement.station, Measurement.tobs, Measurement.prcp).filter(extract('month', Measurement.date) == '6')


temp_june_df = pd.DataFrame(s_june_results, columns=['Date','Station', 'Temperatures','Precipitation'])
temp_june_df


,Date,Station,Temperatures,Precipitation
0,2010-06-01,USC00519397,78.0,0.00
1,2010-06-02,USC00519397,76.0,0.01
2,2010-06-03,USC00519397,78.0,0.00
3,2010-06-04,USC00519397,76.0,0.00
4,2010-06-05,USC00519397,77.0,0.00
...,...,...,...,...
1695,2017-06-26,USC00516128,79.0,0.02
1696,2017-06-27,USC00516128,74.0,0.10
1697,2017-06-28,USC00516128,74.0,0.02
1698,2017-06-29,USC00516128,76.0,0.04


In [21]:
# Calculate the minimum, average, maximum temperatures for each station for June throuout the years. 

stationdata_june_df = temp_june_df.groupby(temp_june_df['Station']) \
                            .agg(min_temp_june = ('Temperatures', 'min'), \
                                 mean_temp_june = ('Temperatures', 'mean'),
                                 max_temp_june = ('Temperatures', 'max'))

print(stationdata_june_df)

             min_temp_june  mean_temp_june  max_temp_june
Station                                                  
USC00511918           66.0       74.139394           85.0
USC00513117           68.0       74.050847           83.0
USC00514830           67.0       76.005376           81.0
USC00516128           66.0       71.937220           79.0
USC00517948           64.0       76.655405           82.0
USC00518838           67.0       73.394737           78.0
USC00519281           65.0       73.271186           82.0
USC00519397           70.0       77.559322           84.0
USC00519523           71.0       76.668103           82.0


In [22]:
# Calculate the minimum, average, maximum precipitation for each station for June throuout the years.

stationdata_june_df = temp_june_df.groupby(temp_june_df['Station']) \
                            .agg(min_prcp_june = ('Precipitation', 'min'), \
                                 mean_prcp_june = ('Precipitation', 'mean'),
                                 max_prcp_june = ('Precipitation', 'max'))

print(stationdata_june_df)

             min_prcp_june  mean_prcp_june  max_prcp_june
Station                                                  
USC00511918            0.0        0.015157           0.58
USC00513117            0.0        0.118248           4.43
USC00514830            0.0        0.114192           3.60
USC00516128            0.0        0.495748           2.80
USC00517948            0.0        0.057975           1.71
USC00518838            0.0        0.094615           0.82
USC00519281            0.0        0.151525           1.39
USC00519397            0.0        0.022661           0.89
USC00519523            0.0        0.050044           0.51


In [23]:
# Retrieve the stations for month of December, save into list and create a DataFrame.

s_dec_results = session.query(Measurement.date,Measurement.station, Measurement.tobs, Measurement.prcp).filter(extract('month', Measurement.date) == '12')
temp_dec_df = pd.DataFrame(s_dec_results, columns=['Date','Station', 'Temperatures','Precipitation'])
temp_dec_df

,Date,Station,Temperatures,Precipitation
0,2010-12-01,USC00519397,76.0,0.04
1,2010-12-03,USC00519397,74.0,0.00
2,2010-12-04,USC00519397,74.0,0.00
3,2010-12-06,USC00519397,64.0,0.00
4,2010-12-07,USC00519397,64.0,0.00
...,...,...,...,...
1512,2016-12-27,USC00516128,71.0,0.14
1513,2016-12-28,USC00516128,71.0,0.14
1514,2016-12-29,USC00516128,69.0,1.03
1515,2016-12-30,USC00516128,65.0,2.37


In [24]:
#Calculate the minimum, average, maximum temperatures for each station for December throuout the years. 

stationdata_dec_df = temp_dec_df.groupby(temp_dec_df['Station']) \
                            .agg(min_temp_dec = ('Temperatures', 'min'), \
                                 mean_temp_dec = ('Temperatures', 'mean'),
                                 max_temp_dec = ('Temperatures', 'max'))

print(stationdata_dec_df)

             min_temp_dec  mean_temp_dec  max_temp_dec
Station                                               
USC00511918          57.0      69.684211          81.0
USC00513117          63.0      71.069444          80.0
USC00514830          62.0      73.224719          79.0
USC00516128          60.0      69.291262          83.0
USC00517948          64.0      71.834862          82.0
USC00518838          63.0      72.421053          78.0
USC00519281          58.0      69.903226          79.0
USC00519397          56.0      71.109524          77.0
USC00519523          56.0      72.433333          79.0


In [25]:
#Calculate the minimum, average, maximum precipitations for each station for December throuout the years. 


stationdata_dec_df = temp_dec_df.groupby(temp_dec_df['Station']) \
                            .agg(min_prcp_dec = ('Precipitation', 'min'), \
                                 mean_prcp_dec = ('Precipitation', 'mean'),
                                 max_prcp_dec = ('Precipitation', 'max'))

print(stationdata_dec_df)

             min_prcp_dec  mean_prcp_dec  max_prcp_dec
Station                                               
USC00511918          0.00       0.138146          4.00
USC00513117          0.00       0.203241          4.94
USC00514830          0.00       0.154966          2.55
USC00516128          0.00       0.507005          6.42
USC00517948          0.00       0.152727          2.80
USC00518838          0.02       0.638182          2.50
USC00519281          0.00       0.244931          3.14
USC00519397          0.00       0.075314          2.01
USC00519523          0.00       0.162010          4.95
